In [ ]:
# For Data Reading 
# Import  Data for Spam or Ham 
import numpy as np
def csv_to_numpy_array(filePath, delimiter):
    return np.genfromtxt(filePath, delimiter=delimiter, dtype=None)

def import_data() :
    print ("Loading data...")
    trainX_path = "/home/hyunyoung2/my-jupyter/spam-filter/Project/Project_code_spam/tensorflow-tutorial/data/trainX.csv"
    trainY_path = "/home/hyunyoung2/my-jupyter/spam-filter/Project/Project_code_spam/tensorflow-tutorial/data/trainY.csv"
    testX_path = "/home/hyunyoung2/my-jupyter/spam-filter/Project/Project_code_spam/tensorflow-tutorial/data/testX.csv"
    testY_path = "/home/hyunyoung2/my-jupyter/spam-filter/Project/Project_code_spam/tensorflow-tutorial/data/testY.csv"
    trainX = csv_to_numpy_array(trainX_path, delimiter="\t")
    trainY = csv_to_numpy_array(trainY_path, delimiter="\t")
    testX = csv_to_numpy_array(testX_path, delimiter="\t")
    testY = csv_to_numpy_array(testY_path, delimiter="\t")
    
    return trainX, trainY, testX, testY

In [ ]:
## test 2-2 (Activation)
## The number of epoch : 27000
## cost function : cross entropy
## activation function : tanh
## Update Method : GradientDscent
## The number of Hidden_Layer : 1

trainX, trainY, testX, testY = import_data()

print ("Import was done : ")
print ("trainX shape :",trainX.shape)
print ("trainY shape :",trainY.shape)
print ("testX shape :",testX.shape)
print ("testY shape :",testY.shape)

import tensorflow as tf

# Parameter 
traing_epochs = 27000
## Step size = A smarter Learing rate for gradientOptimizer
## Basically, You can use a constant like "learning_rate = 0.0008"
training_learing_rate = 0.0008 #tf.train.exponential_decay(learning_rate=0.0008,
                               #          global_step=1,
                               #          decay_steps=trainX.shape[0],
                               #          decay_rate = 0.95,
                               #          staircase=True)
# Network Parameter
## z = a(W_1*x+b_1)
## Y = W_2*z + b_2
n_hidden_1 = trainX.shape[1] # Currently, the size of Word2Vec is 300
                               # when you know about word2Vec size like the above "n_hidden"
logs_path = '/home/hyunyoung2/my-jupyter/tensorflow/test_example/HCTL_2017_About_DeepLearning_2/3rd_copy/'

n_inputs = trainX.shape[1] # In other words, Input size is 300 
print ("n_inputs :", n_inputs)
## Currently, n_label is a couple like two-dimension [0, 1]
## But later on, you can change the type of classes like below
## "n_classes" 
n_labels = trainY.shape[1]
print ("n_labels : ",  n_labels)

# tf Graph input : inputs and ground_truth
x = tf.placeholder(tf.float32, [None, n_inputs])
ground_truth = tf.placeholder(tf.float32, [None, n_labels])

print ("starting store layers... ")
# Store layers weights & bias
weights = {
    "h1" : tf.Variable(tf.random_normal([n_inputs, n_hidden_1], mean=0,
                                        stddev=(np.sqrt(6/(n_inputs + n_hidden_1+1))),
                                        name = "weight_h1")),
   
    "out" : tf.Variable(tf.random_normal([n_hidden_1, n_labels], mean=0,
                                        stddev=(np.sqrt(6/(n_hidden_1 + n_labels+1))),
                                        name = "weight_out"))
}

biases = {
    "b1" : tf.Variable(tf.random_normal([1, n_hidden_1],
                                        mean=0,
                                        stddev=(np.sqrt(6/(n_inputs + n_hidden_1+1))),
                                        name = "bias_h1")),
    
    "out" : tf.Variable(tf.random_normal([1, n_labels],
                                         mean=0,
                                         stddev=(np.sqrt(6/(n_hidden_1 + n_labels+1))),
                                         name = "bias_out"))                                   
}

print("Starting Fully_connected_NN...")
# Create Model
def multilayer_fully_connected_NN(x, weight, biases) :
    # Hidden layer with Sigmoid Activation
    layer_1_biases = tf.add(tf.matmul(x, weight["h1"]), biases["b1"])
    print ("layer_1_biases : ", layer_1_biases.shape)
    layer_1 = tf.nn.tanh(layer_1_biases)
    
    out_layer_final = tf.add(tf.matmul(layer_1, weight["out"]), biases["out"])
    print ("weight['out'] :", weight["out"].shape)
    print ("out_layer_final : ", out_layer_final.shape)
    #out_layer = tf.nn.sigmoid(out_layer_biases)
    #print ("out_layer :", out_layer.shape)
    
    return out_layer_final

# Predict with a model you contruct 
training_prediction = multilayer_fully_connected_NN(x, weights, biases)
# Construct model

pred = tf.nn.softmax(training_prediction) # Softmax

# Define loss and optimizer 
#training_cost = tf.nn.l2_loss((training_prediction-ground_truth), name="squared_error_cost")
training_cost = tf.reduce_mean(-tf.reduce_sum(ground_truth*tf.log(pred), reduction_indices=[1]))
training_optimzer = tf.train.GradientDescentOptimizer(learning_rate=training_learing_rate).minimize(training_cost)
#training_optimzer = tf.train.AdamOptimizer(learning_rate=training_learing_rate).minimize(training_cost)

# For printing the intermdediate data
correct_prediction_OP = tf.equal(tf.argmax(training_prediction,1), tf.argmax(ground_truth,1))
accuracy_OP = tf.reduce_mean(tf.cast(correct_prediction_OP, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", training_cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", accuracy_OP)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()


print("Starting Session...")
# Launch The tensorflow graph
with tf.Session() as sess :
    sess.run(init)
    
     # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    # training cycle
    for i in range(traing_epochs) :
        step, summary =  sess.run([training_optimzer, merged_summary_op], feed_dict={x : trainX, ground_truth: trainY})
        # Write logs at every iteration
        summary_writer.add_summary(summary, i)
        if i % 100 == 0 : 
            train_accuracy, newCost = sess.run(
                [accuracy_OP, training_cost], 
                feed_dict={x: trainX, ground_truth: trainY})
            print("step %d, training accuracy %g"%(i, train_accuracy))
            print("step %d, cost %g"%(i, newCost))
    
    # Finally, check Test data
    print ("Final Accuracy on Test set: %s" % str(sess.run(accuracy_OP, feed_dict={x: testX, ground_truth: testY})))